In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from utility import get_output_path

plt.style.use('ggplot')
pd.set_option('display.float_format', '{:.2f}'.format)

compare_items = [
    'end_balance',
    'total_days',
    'profit_days',
    'total_trade_count',
    'max_ddpercent',
    'max_drawdown_duration',
    'total_return',
    'annual_return',
    'return_std',
    'daily_return',
    'sharpe_ratio',
    'return_drawdown_ratio'
]

In [ ]:
# 成交结果对比分析
cnt_items = [item + '_cnt' for item in compare_items]
seg_items = [item + '_seg' for item in compare_items]
keep_item = cnt_items + seg_items
keep_item.insert(0, 'name')

fp = get_output_path('batch_result.csv')
df = pd.read_csv(fp, index_col=0)
df = df[keep_item]

In [ ]:
for name in compare_items:
    df[name] = df[name + '_cnt'] - df[name + '_seg']
keep_items = compare_items[:]
keep_items.insert(0, 'name')
df = df[keep_items]
df.to_csv(get_output_path('processed.csv'))
df.describe()

In [ ]:
df.skew()

In [ ]:
df.kurt()

In [ ]:
%matplotlib notebook

%matplotlib notebook
plt.figure(figsize=(10, 20))

balance_plot = plt.subplot(5, 1, 1)
balance_plot.set_title("Total Return")
df["total_return"].hist(bins=50)

drawdown_plot = plt.subplot(5, 1, 2)
drawdown_plot.set_title("Drawdown Percent")
df["max_ddpercent"].hist(bins=50)

daily_return_plot = plt.subplot(5, 1, 3)
daily_return_plot.set_title("Daily Return")
df["daily_return"].hist(bins=50)

return_std_plot = plt.subplot(5, 1, 4)
return_std_plot.set_title("Daily Return Std")
df["return_std"].hist(bins=50)

return_std_plot = plt.subplot(5, 1, 5)
return_std_plot.set_title("Sharpe Ratio")
df["sharpe_ratio"].hist(bins=50)

plt.tight_layout() 

### 单项结果对比

In [ ]:
# test_folder = Path(r'D:\research_tools\backtest\result\sc_turtle_entry_window_25.exit_window_10')
# test_folder = Path(r'D:\research_tools\backtest\result\sc_turtle_entry_window_25.exit_window_10.fixed_size_1')
test_folder = Path(r'E:\project\research_tools\backtest\result\sc_turtle_entry_window_25.exit_window_10.fixed_size_1')

trade_kwargs = {'index_col': 0, 'parse_dates': True}
pnl_kwargs = {'index_col': 0, 'parse_dates': True, 'usecols': [0, 1, 2, 4, 5, 6, 10, 11, 12, 13]}
result_kwargs = {'index_col': 0}
trade_cont = pd.read_csv(test_folder.joinpath('trade_continuous.csv'), **trade_kwargs)
trade_seg = pd.read_csv(test_folder.joinpath('trade_seg.csv'), **trade_kwargs)
pnl_cont = pd.read_csv(test_folder.joinpath('pnl_continuous.csv'), **pnl_kwargs)
pnl_seg = pd.read_csv(test_folder.joinpath('pnl_seg.csv'), **pnl_kwargs)
res_cont = pd.read_csv(test_folder.joinpath('result_continuous.csv'), **result_kwargs)
res_seg = pd.read_csv(test_folder.joinpath('result_seg.csv'), **result_kwargs)
res_cont = res_cont[compare_items]
res_seg = res_seg[compare_items]

In [ ]:
test_folder = Path(r'E:\project\research_tools\backtest\result\sc_turtle_entry_window_25.exit_window_10.fixed_size_1')
pnl_seg = pd.read_csv(test_folder.joinpath('pnl_seg.csv'), index_col=0, parse_dates=True)
pnl_seg.head()

In [ ]:
pnl_seg.tail()
# pnl_seg['trade_count'].sum()
t_df = pnl_seg.iloc[-1:].copy()
# t_df.iloc[0]['close_price'] = 999
last_index = t_df.iloc[0].name
t_df.loc[last_index, 'close_price'] = 888
t_df

In [ ]:
def merge_duplicate_date(df):
    """Groud process function"""
#     print(df.index)
#     df = df.copy()
    if len(df) == 1:
#         print(df.index)
        return df.iloc[-1:].copy()
    else:
        add_items = ['trade_count', 'turnover', 'commission', 'slippage', 'trading_pnl', 'holding_pnl', 'total_pnl', 'net_pnl']
        df2 = df.iloc[-1:].copy()
#         print(df2)
        last_index = df2.iloc[0].name
        df2.loc[last_index, 'start_pos'] = df.iloc[0]['start_pos']
        for item in add_items:
            df2.loc[last_index, item] = df[item].sum()
        print(df2.index)
        return df2

new_pnl = pnl_seg.groupby(pnl_seg.index, group_keys=False).apply(merge_duplicate_date)
new_pnl.index

In [ ]:
new_pnl

In [ ]:
new_pnl.to_csv(test_folder.joinpath('new_pnl.csv'))

In [ ]:
trade_comp = trade_cont.merge(trade_seg, left_index=True, right_index=True, how='outer', suffixes=('_cont','_seg'))
pnl_comp = pnl_cont.merge(pnl_seg, left_index=True, right_index=True, how='outer', suffixes=('_cont','_seg'))
res_comp = res_cont.merge(res_seg, left_index=True, right_index=True, how='outer', suffixes=('_cont','_seg'))

pnl_comp.to_csv(test_folder.joinpath('pnl_comp.csv'))
trade_comp.to_csv(test_folder.joinpath('trade_comp.csv'))

In [ ]:
# trade_comp.head()
# 选取拼接合约成交点位和分段回测不同的单
df = trade_comp[trade_comp.isnull().T.any()]
df
# num = len(df[df['direction_cont'].notnull()])
# num / len(trade_cont)

In [ ]:
df = pnl_comp[pnl_comp.isnull().T.any()]
df

In [ ]:
# 每日盈亏数据不一致的记录
pnl_comp.head()
df_diff = pnl_comp[pnl_comp['net_pnl_cont'] != pnl_comp['net_pnl_seg']]
df_diff
len(df_diff) / len(pnl_seg)

In [ ]:
# 选取成交点位相同但是方向不一致的单
df = trade_comp[~trade_comp.isnull().T.any()]
df[df.direction_cont != df.direction_seg]